In [1]:
%matplotlib inline
import matplotlib

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import discopt
import pandas as pd
import numpy as np
import copy
import sklearn
import sklearn.datasets
import xgboost as xgb
import scipy
from scipy import stats

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
n_jobs = 3
ntrials = 30

# Prepare data 

In [5]:
np.random.seed(0)
digits = sklearn.datasets.load_digits()
x = digits.data
y = digits.target

ntrain = 100
order = np.random.permutation(y.size)
tr = order[:ntrain]
val = order[ntrain:]
ytr = y[tr]
xtr = x[tr, :]
yval = y[val]
xval = x[val, :]


# DISCOPT SHORTCUT FOR XGB 

In [6]:
# parameter space is proposed in discopt.py file

In [7]:
# write your train_and_valid function
def train_and_valid(clf):
    # put here trainig and validation, return objective
    # optimizer would try to minimize this value, so use NEGATIVE accuracy/AUC/F1...
    # validation method is up to you
    # clf is xgboost.sklearn.XGBoostClassifier instance
    clf.fit(xtr, ytr)
    pred = clf.predict(xval)
    obj = -(pred == yval).mean() # (minus) accuracy
    return obj

In [8]:
best_params, scores = discopt.search_params_for_xgb(train_valid_func=train_and_valid,
                                                    train_data_shape=xtr.shape,
                                                    ntrials=ntrials,
                                                    n_jobs=n_jobs)

best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.861520329994
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.870359457867
best objective yet -0.872127283441
best objective yet -0.872127283441
best objective yet -0.872127283441
best objective yet -0.872127283441
best objective yet -

In [13]:
# the long 'stagnations' of 'best value so far' is typical for every optimizer, at least for this particular problem

In [14]:
best_params

{'colsample_bylevel': 0.58285755492823554,
 'colsample_bytree': 0.43000199503194431,
 'gamma': 8.8111125567087836,
 'learning_rate': 0.015799103981259305,
 'max_depth': 12,
 'n_estimators': 294,
 'reg_lambda': 2.0613093422711053,
 'subsample': 0.82661228527714992}

In [15]:
# dataframe with objective and 
scores.head()

,lr_trees_ratio,colsample_bytree,colsample_bylevel,max_depth,n_estimators,subsample,log_reg_lambda,log_gamma,obj
0,6.960429,0.667547,0.323762,11.0,259.0,0.212961,1.866324,-2.268405,-0.707720
1,3.235117,0.260091,0.358578,5.0,290.0,0.937476,-2.027846,-2.073616,-0.861520
2,5.526780,0.510742,0.909925,5.0,161.0,0.379375,-2.184782,1.380114,-0.752504
3,6.419523,0.608385,0.619049,20.0,248.0,0.356883,-1.358569,0.193842,-0.832646
4,3.822966,0.324387,0.235085,13.0,56.0,0.475294,-1.882403,-4.482429,-0.830878
